# Lesson 4 - User-ID

<img src=".images/userid-overview.png" alt="User-ID Use Cases" style="width: 750px;"/>

## What is User-ID?

**User-ID** is not just about users. From an API perspective, User-ID encompases any *dynamic* configuration change to the firewall, ie. any configuration change to the firewall that doesn't require a **commit**. A dynamic change can include any of the following:

| User-ID Function               | Description                                                         |
|--------------------------------|---------------------------------------------------------------------|
| User Login / Logout            | Creates and removes User-to-IP mappings                             |
| User / Group Membership        | Often retrieved from AD, you can supplement group info with the API |
| Host Information Profile (HIP) | Information about the OS, disk encryption, antivirus, etc of a host |
| Dynamic Address Groups         | Tag IP addresses with metadata and enforce policy on tags           |

In this lab we'll cover **User Login/Logout** and **Dynamic Address Groups**.

User and Server information (User-ID information) is often collected natively from external systems without using the API. Such external systems might include an Active Directory Server, Wireless AP Controller, Virtual Desktop Manager, Virtualization Orchestrator, or AWS Public Cloud. The API is used for systems that are not natively supported, such as a proprietary app, non-standard Radius server, or VPN concentrator. You can send updates to the firewall from these system using the User-ID API. Device Framework makes this very easy.

## Login and Logout

The next-generation firewall can enforce policy based on users and groups, instead of IP addresses. To do this, the firewall must always know who is logged into each computer. The firewall maintains a table of User to IP mappings, so when it gets network traffic from an IP, it knows which user generated this traffic.

You can create and remove entries in the User to IP mapping table using the API. Here's an example of adding an entry:

In [ ]:
# First, import the needed module and create a Firewall object to work with
from pandevice import firewall
fw = firewall.Firewall('10.30.11.101', 'admin', 'Ignite18')

In [ ]:
# Now, use the Firewall object's userid subsystem to create a mapping
fw.userid.login('example\\bob', '10.0.1.10', timeout=60)

Note: The **timeout** argument is optional and indicates the time in minutes that the entry will exist in the table.

Validate the API call worked by logging into the CLI of the firewall. To do this, you'll need to SSH into the Ubuntu server, then SSH from there to the private IP of the firewall. Use the credentials in the code above to login.

Once connected to the firewall CLI, run this command to view the User to IP mapping table:  
`show user ip-user-mapping all`  (this is a firewall command, not a python command)

You should see the user mapping on the firewall now.

The entry in the table will timeout after 60 minutes, but say Bob logged out before 60 minutes. Delete the entry in the User to IP mapping table:

In [ ]:
fw.userid.logout('example\\bob', '10.0.1.10')

Validate again using the firewall CLI to ensure the entry was removed from the table.

## Dynamic Address Groups

Dynamic Address Groups are a powerful tool for enforcing security policy. They allow the firewall's security policy to be dynamically kept up-to-date with knowledge from external systems such as orchestration, virtualization, cloud, IPAM, NAC, and other tools. For example, the firwall natively supports pulling metadata about virtual machines from VMware vCenter and AWS. There are also integrations with OpenStack, Cisco ACI, and many others to gather metadata for policy use. The firewall is aware of the guest OS, UUID, name, and purpose of the virtual machines so you can create policy based on these attributes, instead of IP addresses.

You can also add metadata to the firewall using the API. This is useful in cases where the orchestration tool isn't already integrated with the firewall, or you want to build security policy on additional information not natively supported such as tenant metadata shared to the firewall.

There are two parts to a Dynamic Address Group:

1. The Dynamic Address Group and match criteria
2. The Registered IP and tag table

The **Dynamic Address Group** and match criteria is just like a regular Address Group, but instead of giving it a list of IP addresses, you give it a match criteria consisting of tags.  For example, a dynamic address group might match on: `'newyork' AND 'linux' AND 'web'` indicating that the IP addresses in this address group are linux-based web servers located in New York. You can now use this address group in a policy, and whenever a new linux web server is spun up in New York, it will automatically be part of this address group.

The **Registered IP and tag table** is a table in the firewall similar to the User to IP mapping table, but instead of mapping IPs to users, it maps IPs to tags. These tags are the same tags used in the match criteria in the Dynamic Address Group. So if we follow the same example, any IP address registered with all 3 tags ('newyork', 'linux' and 'web') will be included in the linux New York web server Dynamic Address Group.

Let's try it. First, create a Dynamic Address Group, and add it to a policy:

In [ ]:
from pandevice import firewall, objects, policies
fw = firewall.Firewall('10.30.11.101', 'admin', 'Ignite18')

In [ ]:
# Create a Dynamic Address Group and add it as a child of the Firewall in one line
ny_web = fw.add(objects.AddressGroup('New York Linux Web Servers',
                                     dynamic_value="'newyork' AND 'linux' AND 'web'",
                                     description="My first Dynamic Address Group"))

In [ ]:
ny_web.create()

Check the firewall GUI under **Objects -> Address Groups** to verify the Dynamic Address Group was created.

Then create a security policy to use this address group. For a Dynamic Address Group to populate with IP addresses, it must be used in a security policy and committed.

In [ ]:
# Create a security rule
rulebase = fw.add(policies.Rulebase())
ny_web_rule = rulebase.add(policies.SecurityRule('New York web servers allowed',
                                                 fromzone='untrust',
                                                 tozone='trust',
                                                 destination=ny_web,
                                                 application=['web-browsing', 'ssl'],
                                                 action='allow',
                                                 description='Allow web traffic to the New York web servers',
                                                ))

In [ ]:
ny_web_rule.create()

In [ ]:
# Commit the new security rule and dynamic address group
fw.commit(sync=True)

That's half the formula, you've create a dynamic address group. But it won't populate with IP addresses unless you satisfy its match criteria with some registered IP tags. You can see the dynamic address group is empty by navigating in the firewall GUI to **Objects -> Address Groups**. Then, in the row for your dynamic address group, click **more...** to see the IP addresses currently populating this group. It should be empty because you haven't created any registered IP tags yet.

Tag some IP addresses now using the userid subsystem, similar to the `login` method used in the last section:

In [ ]:
fw.userid.register('10.0.5.5', ['newyork', 'linux', 'web'])
fw.userid.register('10.0.6.6', ['london', 'linux', 'database'])

On the firewall CLI, use this command to validate the registered IP tags were created:  
`show object registered-ip all`

Note that IP addresses can have any number of tags, and the tags can be any string that makes sense to you. The tags must match the criteria in the dynamic address group to be included in the group.

**Validate** the tags were picked up by the Dynamic Address Group.  Navigate in the firewall GUI to **Objects -> Address Groups**. Then, in the row for your dynamic address group, click **more...** to see the IP addresses currently populating this group. IMPORTANT: Dynamic Address Groups are repopulated every 60 seconds, so it can take between 0 and 60 seconds for a new registered IP to show up in the dynamic address group in the GUI. New registered IPs will always show on the CLI immediately.

You should see 10.0.5.5 because it matches the Dynamic Address Group criteria, but you will not see 10.0.6.6 because it doesn't match the criteria.

Tags can be removed using the `unregister()` method, which is similar in usage to the `register()` method demostrated here.

## Batch User-ID Operations

In a large network, User-ID operations can be very rapid. Users login and logout constantly, and VMs are spun up and torn down all day. If you made an API call for every one of these events, you could easily overwhelm the firewall's management plane with API calls. That's why the User-ID API allows multiple events to be batched together in a single API call.

Use the `batch_start()` and `batch_end()` methods to control the batching of User-ID operations. Any User-ID method that is called after a `batch_start()` will not result in an API call. Instead, the operation is added to the batch. The `batch_end()` method will close out the batch and send the batched operations collected so far to the firewall.

In [ ]:
fw.userid.batch_start()
fw.userid.register('10.0.7.7', 'mongodb')
fw.userid.register('10.0.8.8', 'apache')

After executing that cell, check the registered IP tag table with `show object registered-ip all` and notice that the entries above were **not** created on the firewall. That's because they were added to the batch instead. End the batch to send the collected operations to the firewall.

In [ ]:
fw.userid.batch_end()

Check the registered IP tag table again to verify the two new entries were created. Both registered IP entries were made using the same API call.

## Exercise 1: Login a user

Use the User-ID subsystem to tell the firewall that Jill logged into a computer at 10.0.1.11. Jill's username is `mycompany\\jill`. Ensure the login entry times out after 40 minutes.

In [ ]:
fw.userid.login('mycompany\\jill', '10.0.1.11', timeout=40)

In [ ]:
# Exercise 1 answer here...


**Validate**: Use the firewall CLI to validate the User to IP mapping was created.

## Exercise 2: Unregister a tag

There were several registered IP tags created in the example steps above, including one server with IP 10.0.5.5 that was tagged with `newyork`, `linux`, and `web` so it would be included in the **New York Linux Web Servers** dynamic address group.  This server is no longer running on linux, so remove only the `linux` tag from this registered IP address.

In [ ]:
fw.userid.unregister('10.0.5.5', 'linux')

In [ ]:
# Exercise 2 answer here...


**Validate**: Use the firewall CLI to verify the tag `linux` was removed. The `newyork` and `web` tags should remain. Verify in the firewall GUI that the IP address was automatically removed from the **New York Linux Web Servers** dynamic address group.

## Exercise 3: Multiple operations

Use a User-ID Batch to perform several operations in a single API call. The operations are:

1. User `mycompany\\kurt` logged into 10.0.1.13.
2. User `mycompany\\nancy` logged into 10.0.1.14.
3. User `mycompany\\jill` logged out of 10.0.1.11. (ie. remove the mapping created in exercise 1)
4. Two new Linux Web Servers were spun up in New York with IP addresses 10.0.9.2 and 10.0.9.3, so tag them with these tags: `newyork`, `linux`, and `web`

Extra credit: Register both New York linux web server IPs using a single call to the `register()` method. You might need to reference the documentation for the `register()` method to figure out how.

In [ ]:
fw.userid.batch_start()
fw.userid.login('mycompany\\kurt', '10.0.1.13')
fw.userid.login('mycompany\\nancy', '10.0.1.14')
fw.userid.logout('mycompany\\jill', '10.0.1.11')
fw.userid.register(['10.0.9.2', '10.0.9.3'], ['newyork', 'linux', 'web'])
fw.userid.batch_end()

In [ ]:
# Exercise 3 answer here...







**Validate**: Use the firewall CLI to verify there are two user mappings (Kurt and Nancy). Verify in the firewall GUI that the IP address (10.0.9.2 and 10.0.9.3) were automatically added to the **New York Linux Web Servers** dynamic address group. (Remember it can take up to 60 seconds for the dynamic address group to update)

# Next steps

You've learned how to make dynamic changes to the firewall. This is a powerful feature of the next-generation firewall and much of it is unique to Palo Alto Networks. Leverage the tools you've learned here to improve operational effeciency, reduce change control requirements, and eliminate stale security rules in your organization.

Everything we've done so far has involved only a single firewall. Next, you'll learn how to effectively leverage Panorama to scale out what you've learned across multiple firewalls in your network. Continue on to [Lesson 5 - Panorama](Lesson 5 - Panorama.ipynb).